# Salvando os dados históricos da inflação ao produtor de bebidas no banco de dados.



In [1]:
import pandas as pd
import sqlite3
import json

import configs as cfg
import helpers as hlp

In [2]:
with open(cfg.ARQUIVO_DADOS_IPP_BEBIDAS, 'r') as file:
    json_ipp = json.load(file)

In [3]:
list_data = pd.DataFrame(json_ipp)

list_data.head()

,id,variavel,unidade,resultados
0,1396,IPP - Variação mês/mês imediatamente anterior ...,,"[{'classificacoes': [{'id': '842', 'nome': 'In..."
1,1395,IPP - Variação acumulada no ano (em relação a ...,,"[{'classificacoes': [{'id': '842', 'nome': 'In..."
2,1394,IPP - Variação mês/mesmo mês do ano anterior (...,,"[{'classificacoes': [{'id': '842', 'nome': 'In..."
3,10008,IPP - Número-índice (dezembro de 2018 = 100),Número-índice,"[{'classificacoes': [{'id': '842', 'nome': 'In..."


## Montando Data Frame das variações mês-a-mês

In [4]:
df_mes_a_mes = pd.DataFrame(json_ipp[0]['resultados'][0]['series'][0]['serie'].items())

df_mes_a_mes.columns = ["id_mes","var_mes"]

df_mes_a_mes.head()

,id_mes,var_mes
0,200912,...
1,201001,0.80
2,201002,0.13
3,201003,0.00
4,201004,-0.92


## Montando Data Frame dos acumulados no ano

In [5]:
df_acumulado_ano = pd.DataFrame(json_ipp[1]['resultados'][0]['series'][0]['serie'].items())

df_acumulado_ano.columns = ["id_mes","acum_ano"]

df_acumulado_ano.head()

,id_mes,acum_ano
0,200912,...
1,201001,0.80
2,201002,0.93
3,201003,0.93
4,201004,0.00


## Montando Data Frame dos números índices

In [6]:
df_numero_indice = pd.DataFrame(json_ipp[3]['resultados'][0]['series'][0]['serie'].items())

df_numero_indice.columns = ["id_mes","numero_indice"]

df_numero_indice.head()

,id_mes,numero_indice
0,200912,56.73842
1,201001,57.19060
2,201002,57.26588
3,201003,57.26636
4,201004,56.74100


## Montando o Data Frame único com todos os dados

In [7]:
df_ipp_bebidas = pd.merge(df_numero_indice, df_mes_a_mes, on='id_mes')

df_ipp_bebidas.head()

,id_mes,numero_indice,var_mes
0,200912,56.73842,...
1,201001,57.19060,0.80
2,201002,57.26588,0.13
3,201003,57.26636,0.00
4,201004,56.74100,-0.92


In [8]:
df_ipp_bebidas = pd.merge(df_ipp_bebidas, df_acumulado_ano, on='id_mes')

df_ipp_bebidas.head()

,id_mes,numero_indice,var_mes,acum_ano
0,200912,56.73842,...,...
1,201001,57.19060,0.80,0.80
2,201002,57.26588,0.13,0.93
3,201003,57.26636,0.00,0.93
4,201004,56.74100,-0.92,0.00


# Tratando e criando uma coluna com o acumulado dos últimos 12 meses.

Vou remover a primeira linha e calcular os últimos 12 meses.

In [9]:
df_ipp_bebidas.drop(0, inplace=True)

df_ipp_bebidas.head()

,id_mes,numero_indice,var_mes,acum_ano
1,201001,57.19060,0.80,0.80
2,201002,57.26588,0.13,0.93
3,201003,57.26636,0.00,0.93
4,201004,56.74100,-0.92,0.00
5,201005,56.54790,-0.34,-0.34


In [10]:
hlp.converter_colunas_para_numericos(df_ipp_bebidas,['numero_indice', 'var_mes','acum_ano'], com_tratamento=False)

In [11]:
df_ipp_bebidas['acum_12_meses'] = df_ipp_bebidas['var_mes'].rolling(window=12, min_periods=1).sum()

df_ipp_bebidas.head()

,id_mes,numero_indice,var_mes,acum_ano,acum_12_meses
1,201001,57.19060,0.80,0.80,0.80
2,201002,57.26588,0.13,0.93,0.93
3,201003,57.26636,0.00,0.93,0.93
4,201004,56.74100,-0.92,0.00,0.01
5,201005,56.54790,-0.34,-0.34,-0.33


In [12]:
df_ipp_bebidas.tail()

,id_mes,numero_indice,var_mes,acum_ano,acum_12_meses
154,202210,128.15431,1.54,15.16,17.42
155,202211,129.98458,1.43,16.80,16.33
156,202212,128.61154,-1.06,15.57,14.65
157,202301,135.52478,5.38,5.38,15.61
158,202302,137.95255,1.79,7.26,16.91


In [13]:
df_ipp_bebidas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 1 to 158
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_mes         158 non-null    object 
 1   numero_indice  158 non-null    float64
 2   var_mes        158 non-null    float64
 3   acum_ano       158 non-null    float64
 4   acum_12_meses  158 non-null    float64
dtypes: float64(4), object(1)
memory usage: 7.4+ KB


# Criando tabela com dados dos períodos e juntado ao IPP

In [14]:
with open(cfg.ARQUIVO_DADOS_PERIODO_IPP, 'r') as file:
    json_periodos = json.load(file)

In [15]:
df_periodos = pd.DataFrame(json_periodos)

df_periodos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           159 non-null    object
 1   literals     159 non-null    object
 2   modificacao  159 non-null    object
dtypes: object(3)
memory usage: 3.9+ KB


In [16]:
df_periodos.head()

,id,literals,modificacao
0,200912,"[dezembro 2009, dezembro de 2009]",16/04/2019
1,201001,"[janeiro 2010, janeiro de 2010]",16/04/2019
2,201002,"[fevereiro 2010, fevereiro de 2010]",16/04/2019
3,201003,"[março 2010, março de 2010]",16/04/2019
4,201004,"[abril 2010, abril de 2010]",16/04/2019


In [17]:
df_periodos['descricao'] = df_periodos['literals'].apply(lambda x:x[0])

df_periodos = df_periodos[['id','descricao']]

df_periodos.columns = ['id_mes','descricao_mes']

df_periodos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_mes         159 non-null    object
 1   descricao_mes  159 non-null    object
dtypes: object(2)
memory usage: 2.6+ KB


In [18]:
df_ipp_bebidas = pd.merge(df_periodos, df_ipp_bebidas, on='id_mes')

df_ipp_bebidas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_mes         158 non-null    object 
 1   descricao_mes  158 non-null    object 
 2   numero_indice  158 non-null    float64
 3   var_mes        158 non-null    float64
 4   acum_ano       158 non-null    float64
 5   acum_12_meses  158 non-null    float64
dtypes: float64(4), object(2)
memory usage: 8.6+ KB


# Salvando os dados no banco de dados

In [20]:
conn = sqlite3.connect(cfg.STR_CONN_SQLLITE)

df_ipp_bebidas.to_sql(cfg.TABELA_IPP_HISTORICO, conn, if_exists='replace', index=False)

conn.close()